# Object Detection in Video

In [1]:
import sys
sys.path.append("../components")

from videoinput.filevideoinput import VideoFileInputSource
from layers.videoprocessinglayers import VideoProcessingLayer
from layers.panner import PannerLayer
from layers.resizer import ResizerLayer
from layers.tensorflowobjectdetection import ObjectDetectionLayer,DrawBoundingBoxesLayer
from layers.writer import WriterLayer
from layers.influxdbstats import InfluxDBStatisticsLayer
from layers.elasticsearchstorage import ElasticSearchObjectDetectionIndexer
from layers.zmqpub import ZMQPubLayer
from videoprocessor import VideoProcessor


# Parameters
INPUT_VIDEO_FILENAME = '../../media/bariloche_15_julio_2017.mp4'
OBJECT_DETECTION_SERVICE_ENDPOINT = 'localhost:8500'
OBJECT_DETECTION_MODEL_SPEC = 'faster_rcnn_resnet50_coco'
LABELS = "../../models/labels/coco-spanish.pb.txt"

OUTPUT_VIDEO_WIDTH = 500
OUTPUT_VIDEO_HEIGHT = 500
OUTPUT_VIDEO_FPS = 20.0

ctx = {}

input_source = VideoFileInputSource( filename=INPUT_VIDEO_FILENAME, start_frame=0)
layers = [        
    ResizerLayer(OUTPUT_VIDEO_WIDTH,OUTPUT_VIDEO_HEIGHT),
    ObjectDetectionLayer(
        endpoint = OBJECT_DETECTION_SERVICE_ENDPOINT,
        model_spec_name = OBJECT_DETECTION_MODEL_SPEC        
    ),
    DrawBoundingBoxesLayer(class_names_filename=LABELS),
    InfluxDBStatisticsLayer(),
    ElasticSearchObjectDetectionIndexer(),
    ZMQPubLayer()
] 

Parse file [../../models/labels/coco-spanish.pb.txt] with text format successfully.


In [2]:
vp = VideoProcessor()
vp.process_video( ctx, input_source, layers )

Opening input video
Starting process
Processing frame 1/7440
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Processing frame 2/7440
Processing frame 3/7440
Processing frame 4/7440
Processing frame 5/7440
Processing frame 6/7440
Processing frame 7/7440
Processing frame 8/7440
Processing frame 9/7440
Processing frame 10/7440
Processing frame 11/7440
Processing frame 12/7440
Processing frame 13/7440
Processing frame 14/7440
Processing frame 15/7440
Processing frame 16/7440
Processing frame 17/7440
Processing frame 18/7440
Processing frame 19/7440
Processing frame 20/7440
Processing frame 21/7440
Processing frame 22/7440
Processing frame 23/7440
Processing frame 24/7440
Processi

KeyboardInterrupt: 

In [2]:
!pip install tensorflow-serving-api==1.12.0
!pip install grpcio-tools
!pip install influxdb
!pip install elasticsearch
#!python3 -m grpc_tools.protoc -I=. --python_out=. LabelMap.proto

You should consider upgrading via the 'pip install --upgrade pip' command.
     |################################| 2.4MB 974kB/s eta 0:00:01
ERROR: tensorflow-gpu 1.12.0 has requirement tensorboard<1.13.0,>=1.12.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorboard 1.14.0 has requirement setuptools>=41.0.0, but you'll have setuptools 40.5.0 which is incompatible.
  Found existing installation: grpcio 1.16.0
    Uninstalling grpcio-1.16.0:
      Successfully uninstalled grpcio-1.16.0
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
     |################################| 92kB 662kB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import sys
sys.path.append("../components/")

from videoinput.filevideoinput import VideoFileInputSource
from layers.videoprocessinglayers import VideoProcessingLayer
from layers.panner import PannerLayer
from layers.progress import DisplayProgressLayer
from layers.resizer import ResizerLayer
from layers.tensorflowobjectdetection import ObjectDetectionLayer,DrawBoundingBoxesLayer
from layers.writer import WriterLayer

from videoprocessor import VideoProcessor

In [5]:
class DebugLayer(VideoProcessingLayer):
    def __init__(self):
        VideoProcessingLayer.__init__(self)
        pass
    
    def setup(self, ctx):
        pass
    
    def process(self, ctx):
        plt.figure(figsize = (40,20))
        plt.imshow(ctx["OUTPUT_FRAME"], interpolation="nearest" )
        pass
    
    def release(self, ctx):
        pass

In [6]:
class DrawStatisticsLayer(VideoProcessingLayer):
    def __init__(self):
        VideoProcessingLayer.__init__(self)
        pass
    
    def setup(self, ctx):
        pass
    
    def process(self, ctx):
        cv2.putText(
             ctx["OUTPUT_FRAME"], 
             "%d [%d,%d]" % (
                 ctx["INPUT_CURR_FRAME"]
             ), 
             (5, ctx["OUTPUT_H"]-10), 
             cv2.FONT_HERSHEY_SIMPLEX, 
             0.4, 
             (255, 255, 255)
        )
        pass
    
    def release(self, ctx):
        pass

In [7]:
# !pip3 install influxdb

from influxdb import InfluxDBClient
from datetime import datetime

class InfluxDBStatisticsLayer(VideoProcessingLayer):
    def __init__(self):
        VideoProcessingLayer.__init__(self)
        self.client = InfluxDBClient(host='localhost', port=8086, database="my_application")
        pass
    
    def setup(self, ctx):
        json_body = []
        
        interesting_variables = [
                    "INPUT_FILENAME",
                    "INPUT_MAX_FRAMES",
                    "INPUT_LENGTH"
        ]
        
        for v in interesting_variables:
            json_body.append(
                {
                    "measurement": v,
                    "time": datetime.now(),
                    "fields": {
                        "value": ctx[v]
                    }
                }
            )        
            self.client.write_points(json_body)
    
    def process(self, ctx):
        json_body = []
        
        json_body.append(
            {
                "measurement": "INPUT_CURR_FRAME",
                "time": datetime.now(),
                "fields": {
                    "value": ctx["INPUT_CURR_FRAME"]
                }
            }
        )        
        
        for k,v in ctx["OBJECT_DETECTION_LABELED_OUTPUT_SCORES"].items():
            print(len(v))
            json_body.append(
                {
                    "measurement": k.upper()+"_COUNT",
                    "time": datetime.now(),
                    "fields": {
                        "value": len(v)
                    }
                }
        )        
        
        self.client.write_points(json_body)
    
    def release(self, ctx):
        self.client.close()

In [8]:
# !pip3 install influxdb
from elasticsearch import Elasticsearch

class ElasticSearchObjectDetectionIndexer(VideoProcessingLayer):
    def __init__(self):
        VideoProcessingLayer.__init__(self)
        self.es = Elasticsearch()
    
    def setup(self, ctx):
        pass
    
    def process(self, ctx):
        doc = {
            'frame': ctx["INPUT_CURR_FRAME"],
            'detectedobjs': ctx["OBJECT_DETECTION_LABELED_OUTPUT_ES"],
            'timestamp': datetime.utcnow()
        }
                
        res = self.es.index(index="detected-objects2", doc_type='object-detections2', body=doc)

    def release(self, ctx):
        pass


## Video Test

In [9]:
# Parameters
INPUT_VIDEO_FILENAME = '../../media/bariloche_15_julio_2017.mp4'

OBJECT_DETECTION_SERVICE_ENDPOINT = 'localhost:8500'
OBJECT_DETECTION_MODEL_SPEC = 'faster_rcnn_resnet50_coco'

OUTPUT_VIDEO_FILENAME = '../../media/cocotest.mp4'
OUTPUT_VIDEO_WIDTH = 320
OUTPUT_VIDEO_HEIGHT = 240
OUTPUT_VIDEO_FPS = 20.0


LABELS = "../../models/labels/coco-spanish.pb.txt"

ctx = {}

vp = VideoProcessor()

vp.process_video( 
    ctx,     
    input_source = VideoFileInputSource(
        filename=INPUT_VIDEO_FILENAME, start_frame=0, max_frames=100
    ),    
    layers =  [        
        ObjectDetectionLayer(
            endpoint = OBJECT_DETECTION_SERVICE_ENDPOINT,
            model_spec_name = OBJECT_DETECTION_MODEL_SPEC        
        ),
        
        DrawBoundingBoxesLayer(class_names_filename=LABELS),
        
        InfluxDBStatisticsLayer(),
        #DebugLayer(),
        ElasticSearchObjectDetectionIndexer(),
        DisplayProgressLayer()
    ]
)

Progress: [####################] Frame 100/100
Processing finished


In [10]:
#!pip install grpcio-tools
#!python3 -m grpc_tools.protoc -I=. --python_out=. LabelMap.proto

In [11]:
ctx

{'INPUT_CURR_FRAME': 101,
 'INPUT_FILENAME': '../../media/bariloche_15_julio_2017.mp4',
 'INPUT_FPS': 30.0,
 'INPUT_H': 720,
 'INPUT_LENGTH': 7440,
 'INPUT_MAX_FRAMES': 100,
 'INPUT_START_FRAME': 0,
 'INPUT_W': 1280,
 'OBJECT_DETECTION_LABELED_OUTPUT_BOXES': {'auto': [(0, 372, 216, 543)],
  'colectivo': [(986, 266, 1280, 503)],
  'persona': [(686, 363, 728, 479),
   (540, 352, 596, 510),
   (646, 372, 673, 459),
   (518, 359, 552, 480),
   (352, 350, 392, 501),
   (268, 355, 317, 494),
   (224, 343, 276, 490),
   (879, 384, 908, 457),
   (870, 378, 901, 453),
   (787, 366, 816, 458),
   (461, 352, 500, 474),
   (437, 349, 477, 463),
   (805, 368, 837, 457),
   (190, 347, 216, 451),
   (902, 380, 928, 458),
   (829, 370, 853, 456),
   (892, 377, 920, 454)]},
 'OBJECT_DETECTION_LABELED_OUTPUT_ES': {'auto': [{'box': {'x0': 0,
     'x1': 216,
     'y0': 372,
     'y1': 543},
    'score': 0.9958763122558594}],
  'colectivo': [{'box': {'x0': 986, 'x1': 1280, 'y0': 266, 'y1': 503},
    'score